In [ ]:
#r "nuget: FParsec"
#r "nuget: Bolero"

Installed Packages Bolero, 0.17.25 FParsec, 1.1.1

In [ ]:
[<StructuredFormatDisplay("{StructuredFormatDisplay}")>]
type Json = JString of string
          | JNumber of float
          | JBool   of bool
          | JNull
          | JList   of Json list
          | JObject of Map<string, Json>
         with
            member private t.StructuredFormatDisplay =
                match t with
                | JString s -> box ("\"" + s + "\"")
                | JNumber f -> box f
                | JBool   b -> box b
                | JNull     -> box "null"
                | JList   l -> box l
                | JObject m -> Map.toList m :> obj

In [ ]:
open FParsec

let jnull  = stringReturn "null" JNull
let jtrue  = stringReturn "true"  (JBool true)
let jfalse = stringReturn "false" (JBool false)

let jnumber = pfloat |>> JNumber // pfloat will accept a little more than specified by JSON
                                 // as valid numbers (such as NaN or Infinity), but that makes
                                 // it only more robust

let str s = pstring s

let stringLiteral =
    let escape =  anyOf "\"\\/bfnrt"
                  |>> function
                      | 'b' -> "\b"
                      | 'f' -> "\u000C"
                      | 'n' -> "\n"
                      | 'r' -> "\r"
                      | 't' -> "\t"
                      | c   -> string c // every other char is mapped to itself

    let unicodeEscape =
        /// converts a hex char ([0-9a-fA-F]) to its integer number (0-15)
        let hex2int c = (int c &&& 15) + (int c >>> 6)*9

        str "u" >>. pipe4 hex hex hex hex (fun h3 h2 h1 h0 ->
            (hex2int h3)*4096 + (hex2int h2)*256 + (hex2int h1)*16 + hex2int h0
            |> char |> string
        )

    let escapedCharSnippet = str "\\" >>. (escape <|> unicodeEscape)
    let normalCharSnippet  = manySatisfy (fun c -> c <> '"' && c <> '\\')

    between (str "\"") (str "\"")
            (stringsSepBy normalCharSnippet escapedCharSnippet)

let jstring = stringLiteral |>> JString

// jvalue, jlist and jobject are three mutually recursive grammar productions.
// In order to break the cyclic dependency, we make jvalue a parser that
// forwards all calls to a parser in a reference cell.
let jvalue, jvalueRef = createParserForwardedToRef() // initially jvalueRef holds a reference to a dummy parser

let ws = spaces // skips any whitespace

let listBetweenStrings sOpen sClose pElement f =
    between (str sOpen) (str sClose)
            (ws >>. sepBy (pElement .>> ws) (str "," .>> ws) |>> f)

let keyValue = tuple2 stringLiteral (ws >>. str ":" >>. ws >>. jvalue)

let jlist   = listBetweenStrings "[" "]" jvalue JList
let jobject = listBetweenStrings "{" "}" keyValue (Map.ofList >> JObject)

do jvalueRef := choice [jobject
                        jlist
                        jstring
                        jnumber
                        jtrue
                        jfalse
                        jnull]

let json = ws >>. jvalue .>> ws .>> eof

let parseJsonString str = run json str

// UTF8 is the default, but it will detect UTF16 or UTF32 byte-order marks automatically
let parseJsonFile fileName encoding =
    runParserOnFile json () fileName encoding

let parseJsonStream stream encoding =
    runParserOnStream json () "" stream System.Text.Encoding.UTF8

In [ ]:
open FParsec.CharParsers

let result = parseJsonFile "test.json" System.Text.Encoding.UTF8
// for the moment we just print out the AST
match result with
| Success (v, _, _) -> v
| Failure (msg, err, _) -> failwith msg

Item
"[ { [Event, [(""Anim"", null); (""Audio"", null); (""Message"", null); (""MsgID2Del"", null); (""Photo"", null); (""Stick"", null); (""Video"", null); (""VideoNote"", ...); (""Voice"", null)]]: Key: Event, Value: { [(""Anim"", null); (""Audio"", null); (""Message"", null); (""MsgID2Del"", null); (""Photo"", null); (""Stick"", null); (""Video"", null); (""VideoNote"", ...); (""Voice"", null)]: Item: [ { [Anim, null]: Key: Anim, Value: null }, { [Audio, null]: Key: Audio, Value: null }, { [Message, null]: Key: Message, Value: null }, { [MsgID2Del, null]: Key: MsgID2Del, Value: null }, { [Photo, null]: Key: Photo, Value: null }, { [Stick, null]: Key: Stick, Value: null }, { [Video, null]: Key: Video, Value: null }, { [VideoNote, [(""ChannelUsername"", """"); (""ChatID"", ...); (""DisableNotification"", ...); (""Duration"", ...); (""File"", null); (""FileID"", ""DQACAgIAAxkBAAEiTzJhvK2LAAFBO36Hc5O7OiHXFp8vg_gAAjMVAAKpj-lJj-S8EHfK2NEjBA""); (""FileSize"", ...); (""Length"", ...); (""MimeType"", """"); (""ReplyMarkup"", null); (""ReplyToMessageID"", ...); (""UseExisting"", ...)]]: Key: VideoNote, Value: { [(""ChannelUsername"", """"); (""ChatID"", ...); (""DisableNotification"", ...); (""Duration"", ...); (""File"", null); (""FileID"", ""DQACAgIAAxkBAAEiTzJhvK2LAAFBO36Hc5O7OiHXFp8vg_gAAjMVAAKpj-lJj-S8EHfK2NEjBA""); (""FileSize"", ...); (""Length"", ...); (""MimeType"", """"); (""ReplyMarkup"", null); (""ReplyToMessageID"", ...); (""UseExisting"", ...)]: Item: [ [ChannelUsername, """"], [ChatID, 215296975.0], [DisableNotification, false], [Duration, 0.0], [File, null], [FileID, ""DQACAgIAAxkBAAEiTzJhvK2LAAFBO36Hc5O7OiHXFp8vg_gAAjMVAAKpj-lJj-S8EHfK2NEjBA""], [FileSize, 0.0], [Length, 384.0], [MimeType, """"], [ReplyMarkup, null], [ReplyToMessageID, 0.0], [UseExisting, true] ] } }, { [Voice, null]: Key: Voice, Value: null } ] } }, { [MessageID, 2248533.0]: Key: MessageID, Value: { 2248533.0: Item: 2248533 } }, { [SessionName, ""Magenta Solution""]: Key: SessionName, Value: { ""Magenta Solution"": Item: Magenta Solution } } ]"
